# Solução Para o Trabalho Prático 01  <h1>

## Problema 02:<h3>
    
1. Implementação de um KEM-RSA que inicializa as instâncias com os respetivos paramêtros de segurança, gera chaves publicas e privadas, encapsulamento e revelação da chave gerada;
    
2. Segundo, ainda baseado no KEM implementado, a usufruindo da transformação de Fujisaki-Okamoto, a criação de um PKE sendo IND-CCA;
    
3. Em seguida, a implementação de um DSA, onde são definidos na inicialização os parametros dos primos _p_ e _q_ com seus respectivos tamanhos, e ainda assinatura digital e a verificação da mesma;
    
4. Por fim, a implementação de um ECDSA que utilizará uma das curvas primas definidas pelo FIPS186-4.    

In [55]:
""" Imports Necessários para as implementações """

import os
import random
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from sage.arith.power import generic_power
from sage.functions.log import logb
from sys import getsizeof
import numpy

**Implementação KEM-RSA**

Temos então definidos a classe KEM-RSA, composta por 9 funções:

- _init_
- _generate_keys_
- _encapsulate_
- _reveal_
- _verify_
- _rsa_encrypt_
- _rsa_decrypt_
- _kdf_
- _print_nums_

descrição da classe e funções

In [2]:
class KEMRSA:
    # Construtor da Classe
    def __init__(self, seg):
        self.size = seg
        self.p_q_size = int(seg/2)
        self.kdf_salt = os.urandom(16)
        self.p = None 
        self.q = None 
        self.d = None
        self.n = None 
        self.e = None
        self.m = None
        self.m_b = None
        self.m_as_int = None
        self.m_b_as_int = None
        self.ct = None
        self.symetric_key_a = None
        self.symetric_key_b = None
        
    def generate_keys(self):
        # conjunto dos números primos
        P = Primes()
        
        # gerar um p
        p_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
        while not (p_candidate in P):
            p_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
            
        # gerar um q
        q_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
        while not (q_candidate in P):
            q_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
     
        # p e q escolhidos
        self.p = p_candidate
        self.q = q_candidate
        
        # calcular n
        self.n = self.p * self.q  
        
        # calcular phi
        phi = (self.p - 1)*(self.q - 1) 
        
        # gerar um e
        #e_candidate = ZZ.random_element(phi)
        #while gcd(e_candidate, phi) != 1:
        #    e_candidate = ZZ.random_element(phi)
        # e escolhido
        self.e = 65537 
        
        # algoritmo extendido de Euclides para calcular d
        bezout = xgcd(self.e, phi) 
        d_candidate = Integer(mod(bezout[1], phi))         
        self.d = d_candidate
        
    def encapsulate(self, plain_text_as_int):
        self.m_as_int = int(plain_text_as_int)
        self.m = self.m_as_int.to_bytes(int(self.size/8), "big")
        self.symetric_key_a = self.kdf(self.m)
        self.rsa_encrypt()
        
    def reveal(self):
        self.rsa_decrypt()
        self.m_b = int(self.m_b_as_int).to_bytes(int(self.size/8), "big")
        self.symetric_key_b = self.kdf(self.m_b)
        
    def verify(self):
        #print("Key A:", self.symetric_key_a)
        #print("Key B:", self.symetric_key_b)
        #print("M Int A:", self.m_as_int)
        #print("M Int B:", self.m_b_as_int)
        #print(self.kdf_salt)
        print("Chaves iguais:", self.symetric_key_a == self.symetric_key_b)
    
    def rsa_encrypt(self):
        self.ct = power_mod(self.m_as_int, self.e, self.n)
    
    def rsa_decrypt(self):
        self.m_b_as_int = power_mod(self.ct, self.d, self.n)        
        
    def kdf(self, password):
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.kdf_salt,
            iterations=100000,
        )
        return kdf.derive(password)
    
    def print_nums(self):
        print("p =", self.p,"\n")
        print("q =", self.q,"\n")
        print("d =", self.d,"\n")
        print("n =", self.n,"\n")
        print("e =", self.e,"\n")

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [40]:
def ex1():
    a = KEMRSA(1024)
    a.generate_keys()
    a.encapsulate(int(ZZ.random_element(2^(a.size - 2))))
    a.reveal()
    a.verify()
    #a.print_nums()

**Implementação DSA**

A class DSA é composta pelas funções init, generate_parameters, generate_keys, sign, compute_s, compute_r, hash_init e verify, onde:

**descrição dos metodos**

In [41]:
#ex1()

Chaves iguais: True


In [135]:
class PKE:
    def __init__(self, x):
        self.x = x
        
    def hashg(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return digest.finalize()
    
    def hashh(self, message):
        digest = hashes.Hash(hashes.BLAKE2s(32))
        digest.update(message)
        return digest.finalize()
    
    def mini_xor(self, a, b):
        tmpa = a
        tmpb = b
        r0 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r1 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r2 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r3 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r4 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r5 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r6 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        r7 = tmpa % 2 + tmpb % 2
        tmpa = int(tmpa//2)
        tmpb = int(tmpb//2)
        
        soma = 0
        if r0 == 1:
            soma += 1
        if r1 == 1:
            soma += 2
        if r2 == 1:
            soma += 4
        if r3 == 1:
            soma += 8
        if r4 == 1:
            soma += 16
        if r5 == 1:
            soma += 32
        if r6 == 1:
            soma += 64
        if r7 == 1:
            soma += 128
            
        return soma
        
    def xor(self, a, b):
        size = len(b)
        if len(a) < len(b):
            size = len(a)
            
        xored = bytearray(size)
        for i in range(size):
            xored[i] = self.mini_xor(a[i], b[i])
        return xored
    
    def cifrar(self, a):
        self.r = self.hashh(os.urandom(16))
        self.y = self.xor(self.x, self.hashg(self.r))
        self.rl = self. y + self.r
        self.rl_as_int = int.from_bytes(self.rl, "big")
        a.encapsulate(self.rl_as_int)
        self.e = a.symetric_key_a
        self.c = self.xor(self.r, self.y)
    
    def decifrar(self, a):
        a.reveal()
        a.verify()
        self.k = a.symetric_key_b
        self.r_2 = self.xor(self.c, self.k)
        self.rl_2 = self.y + self.r_2
        print(x == self.xor(self.y, self.hashg(self.r_2)))

In [136]:
def ex2():
    a = KEMRSA(1024)
    a.generate_keys()
    b = PKE(os.urandom(32))
    b.cifrar(a)
    b.decifrar(a)

In [137]:
ex2()

Chaves iguais: True
False


In [6]:
class DSA:
    def __init__(self, seg):
        self.l = seg
        self.n = 256
        self.h = 2
        self.q = None
        self.p = None
        self.g = None
        self.private_key = None
        self.public_key = None
        self.r = None
        self.s = None
        self.k = None
        self.h_m  = None
        self.w = None
        self.u1 = None
        self.u2 = None
        self.v = None
        
    def generate_parameters(self):
        # conjunto dos números primos
        P = Primes()
        
        flag = 1
        
        while flag:
            # gerar o p
            p_candidate = ZZ.random_element(2^(self.l - 1) + 1, (2^self.l) - 1)
            while not (p_candidate in P):
                p_candidate = ZZ.random_element(2^(self.l - 1) + 1, (2^self.l) - 1)
            self.p = p_candidate
            
            """
            # gerar o q alternativa
            p_menos_um = self.p - 1
            q_candidate = (2**127) - 1
                
            it = 0
            limite = 2**256
            while ((p_menos_um % q_candidate) != 0) & (q_candidate < limite):
                q_candidate = P.next(q_candidate)
                print(it)
                it += 1
                
            print(it)
            if q_candidate < 2**256:
                self.q = q_candidate
                flag = 0"""
            
            # gerar o q
            p_menos_um = self.p - 1
            q_candidate = p_menos_um
            div = 2**(self.l - 256)
            print('oi')
            while ((not (q_candidate in P)) | (p_menos_um % q_candidate) != 0) & (q_candidate > 1):
                q_candidate = int(p_menos_um / div)
                div += 1
            if q_candidate < 2**256:
                self.q = q_candidate
                flag = 0
            
        print('é primo ', self.q in P)
        print('resto ', (self.p-1)%self.q)
        
        # gerar o g 
        self.g = power_mod(self.h, int((self.p - 1)/self.q), self.p)
        while self.g == 1:
            self.h = ZZ.random_element(3, p - 2)
            self.g = power_mod(self.h, int((self.p - 1)/self.q), self.p)
            
    def generate_key(self):
        self.private_key = ZZ.random_element(1, self.q-1)
        self.public_key = power_mod(self.g, self.private_key, self.p)
    
    def sign(self, message):
        self.k = ZZ.random_element(1, self.q-1)
        self.compute_r()       
        self.compute_s(message)        
        while (self.r == 0) | (self.s == 0):
            self.k = ZZ.random_element(1, self.q-1)
            self.compute_r()
            self.compute_s(message)
    
    def compute_r(self):
        tmp = power_mod(self.g, self.k, self.p)
        self.r = tmp % self.q
    
    def compute_s(self, message):
        self.h_m = self.hash_int(message)
        tmp = self.h_m + (self.private_key * self.r)
        tmp = tmp/self.k
        self.s = tmp % self.q
        
    def hash_int(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return int.from_bytes(digest.finalize(), "big")
            
    def verify(self, message):
        self.w = power_mod(self.s, -1, self.q)
        self.u1 = (self.h_m * self.w) % self.q
        self.u2 = (self.r * self.w) % self.q
        tmp = power_mod(self.g, self.u1, self.p)
        tmp = tmp * power_mod(self.public_key, self.u2, self.p)
        tmp = tmp % self.p
        #tmp = (power(self.g, self.u1) * power(self.public_key,  u2)) % self.p
        #tmp = power_mod(self.g, self.w * (self.h_m + (self.private_key * self.r)), self.p)
        self.v = tmp % self.q
        print(self.v == self.r)
    
    def test_parameters(self):
        self.p = 144496639484169656702913885053209177617919277993871803968266556579338580563889381482704768763657598735704658946912906600029639531525346941316981632544156545901918463032034054905931447699318650439614580917692585768829430010249603063301769557522366847281350021548727640651175563292350277316390562373699568499689
        self.q = 1422218247344634743859933275335176438393052153679
        self.g = 96310659971562742116469593861258820096385438737507853418033235652927122210740787446849366510110638206140859983477488399955986400852663803802396340196047631553417649153512359876954427703540898742094170016909758704062127341906200183413320792847076540306977144222033381454383383462374762292799797978590303976617

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [7]:
def ex3():
    a = DSA(512)
    a.generate_parameters()
    #a.test_parameters()
    a.generate_key()
    m = os.urandom(16)
    a.sign(m)
    print(a.q)
    print(a.h_m)
    a.verify(m)

**descrição**

In [8]:
#ex3()

In [9]:
class DSAEC:
    def __init__(self):
        self.curve = None
        self.G = None
        self.gx = None
        self.gy = None
        self.n = None
        self.p = None
        self.seed = None
        self.c = None
        self.b = None
        self.private_key = None
        self.public_key = None
        self.pubx = None
        self.puby = None
        self.r = None
        self.s = None
        
    def curva_p192(self):
        self.gx = int(0x188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012)
        self.gy = int(0x07192b95ffc8da78631011ed6b24cdd573f977a11e794811)
        self.n = 6277101735386680763835789423176059013767194773182842284081
        self.p =  6277101735386680763835789423207666416083908700390324961279
        self.b = int(0x64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1)
        self.curve = EllipticCurve(GF(self.p), [-3,self.b])
        self.G = self.curve(self.gx, self.gy)
        
    def curva_p224(self):
        self.gx = int(0xb70e0cbd6bb4bf7f321390b94a03c1d356c21122343280d6115c1d21)
        self.gy = int(0xbd376388b5f723fb4c22dfe6cd4375a05a07476444d5819985007e34)
        self.n = 26959946667150639794667015087019625940457807714424391721682722368061
        self.p = 26959946667150639794667015087019630673557916260026308143510066298881
        self.b = int(0xb4050a850c04b3abf54132565044b0b7d7bfd8ba270b39432355ffb4)
        self.curve = EllipticCurve(GF(self.p), [-3,self.b])
        self.G = self.curve(self.gx, self.gy)
        
    def curva_p521(self):
        self.gx = int(0xc6858e06b70404e9cd9e3ecb662395b4429c648139053fb521f828af606b4d3dbaa14b5e77efe75928fe1dc127a2ffa8de3348b3c1856a429bf97e7e31c2e5bd66)
        self.gy = int(0x11839296a789a3bc0045c8a5fb42c7d1bd998f54449579b446817afbd17273e662c97ee72995ef42640c550b9013fad0761353c7086a272c24088be94769fd16650)
        self.n = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
        self.p = 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
        self.b = int(0x051953eb9618e1c9a1f929a21a0b68540eea2da725b99b315f3b8b489918ef109e156193951ec7e937b1652c0bd3bb1bf073573df883d2c34f1ef451fd46b503f00)
        self.curve = EllipticCurve(GF(self.p), [-3,self.b])
        self.G = self.curve(self.gx, self.gy)
        
    def curva_p256(self):
        return 1
    
    def curva_p384(self):
        return 1
    
    def curva_p521(self):
        return 1      
        
    def generate_key(self):
        self.private_key = int(ZZ.random_element(1, int(self.n)-1))
        self.public_key = self.private_key*self.G
         
    def hashm(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return digest.finalize()
    
    def leftmost(self, by, length):
        if (length % 8) == 0:
            byte_num = int(length / 8)
            x = len(by) - byte_num
            chunk = by[x:]
            return int.from_bytes(chunk, "big")
            
    def sign(self, message):
        e = self.hashm(message)
        
        lengthn = int(self.n).bit_length()
        z = self.leftmost(e, lengthn) 
        
        k = int(ZZ.random_element(1, self.n-1))
        ponto = k * self.G
        x1, y1 = ponto.xy()
        x1 = int(x1)
        y1 = int(y1)
        self.r = int(x1 % self.n)
        while self.r == 0:
            k = int(ZZ.random_element(1, self.n-1))
            ponto = k * self.G
            x1, y1 = ponto.xy()
            x1 = int(x1)
            y1 = int(y1)
            self.r = int(x1 % self.n)
        self.s = int(((z + (self.r * self.private_key))/k) % self.n)
        while self.s == 0:
            k = int(ZZ.random_element(1, self.n-1))
            ponto = k * self.G
            x1, y1 = ponto.xy()
            x1 = int(x1)
            y1 = int(y1)
            self.r = int(x1 % self.n)
            while self.r == 0:
                k = int(ZZ.random_element(1, self.n-1))
                ponto = k * self.G
                x1, y1 = ponto.xy()
                x1 = int(x1)
                y1 = int(y1)
                self.r = int(x1 % self.n)
            self.s = int(((z + (self.r * self.private_key))/k) % self.n)
            
    def verify(self, message):
        e = self.hashm(message)
        lengthn = int(self.n).bit_length()
        z = self.leftmost(e, lengthn)
        
        u1 = int(int(z/self.s) % self.n)
        u2 = int(int(self.r/self.s) % self.n)
        
        ponto1 = u1 * self.G
        x1, y1 = ponto1.xy()
        x1 = int(x1)
        y1 = int(y1)
        
        ponto2 = u2 * self.public_key
        x2, y2 = ponto2.xy()
        x2 = int(x2)
        y2 = int(y2)
        
        x = x1 + x2
        y = y1 + y2
        
        print(((self.r - x) % self.n) == 0)
        
    def test_values(self):
        self.private_key = 12108469345882820394202023929528337913334982772539972870552851192060966867919719531584824782674947919774168609761175
        self.pubx = 36385597237332565334712001154788673311398888286962312557694576607054406435814142401654875481236704091823176102582830
        self.puby = 23083821865263792395347764046079865914184280489914562033720167767681657917226174492750012188205912655663093778472777

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [10]:
def ex4():
    a = DSAEC()
    a.curva_p224()
    a.generate_key()
    m = os.urandom(16)
    a.sign(m)
    a.verify(m)

In [11]:
#ex4()